# Custom Depth-to-Image Model Playground

#### made by [なんか](https://twitter.com/_determina_)

---

### This notebook does:
  - Perform Task Operation for adapting depth-to-image model to specified domain
  - Play with the model (powered by [AUTOMATIC1111's WebUI](https://github.com/AUTOMATIC1111/stable-diffusion-webui))


---


### THIS NOTEBOOK FORCES RUNTIME TO CRASH AT **SECTION 3.1**!!! 
### IF YOU RUN CELL ALL AT FIRST, RUN SECTION 3.2 AFTER CRASHING. IT IS 100% INTENDED BEHAVIOR.

# 0. Allocate GPU

In [1]:
!nvidia-smi

Fri Jan 20 06:23:54 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    26W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 1. Setup

## 1.1 Install Requirements

In [2]:
!pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip install pytorch_lightning tensorboard omegaconf einops taming-transformers transformers kornia test-tube matplotlib pandas
!pip install diffusers invisible-watermark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining taming-transformers from git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
  Updating ./src/taming-transformers clone (to revision master)
  Running command git fetch -q --tags
  Running command git reset --hard -q 3ba01b241669f5ade541ce990f7650a3b8f65318
  Preparing metadata (setup.py) ... done
  Attempting uninstall: taming-transformers
    Found existing installation: taming-transformers 0.0.1
    Can't uninstall 'taming-transformers'. No files were found to uninstall.
  Running setup.py develop for taming-transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import os
import gc
import copy
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 1.2 Clone AUTOMATIC1111's WebUI

In [4]:
%cd /content/
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui
%cd /content/stable-diffusion-webui

/content
fatal: destination path 'stable-diffusion-webui' already exists and is not an empty directory.
/content/stable-diffusion-webui


# 2. Task Arithmetic

## 2.1 Download Model Weights (base + task-specified models)

### Base model & depth model
- Stable Diffusion V2.0 base
- Stable Diffusion V2.0 depth

In [5]:
!wget https://huggingface.co/stabilityai/stable-diffusion-2-base/resolve/main/512-base-ema.ckpt \
  -O /content/stable-diffusion-webui/models/Stable-diffusion/512-base-ema.ckpt
!wget https://huggingface.co/stabilityai/stable-diffusion-2-depth/resolve/main/512-depth-ema.ckpt \
  -O /content/stable-diffusion-webui/models/Stable-diffusion/512-depth-ema.ckpt

--2023-01-20 06:24:18--  https://huggingface.co/stabilityai/stable-diffusion-2-base/resolve/main/512-base-ema.ckpt
Resolving huggingface.co (huggingface.co)... 3.231.67.228, 54.235.118.239, 2600:1f18:147f:e800:671:b733:ecf3:a585, ...
Connecting to huggingface.co (huggingface.co)|3.231.67.228|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/18/fc/18fcc3be3bc3077cb1e70baea3953cce46fc54fc20aeba602201371707b7c2ed/d635794c1fedfdfa261e065370bea59c651fc9bfa65dc6d67ad29e11869a1824?response-content-disposition=attachment%3B%20filename%3D%22512-base-ema.ckpt%22&Expires=1674451738&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzE4L2ZjLzE4ZmNjM2JlM2JjMzA3N2NiMWU3MGJhZWEzOTUzY2NlNDZmYzU0ZmMyMGFlYmE2MDIyMDEzNzE3MDdiN2MyZWQvZDYzNTc5NGMxZmVkZmRmYTI2MWUwNjUzNzBiZWE1OWM2NTFmYzliZmE2NWRjNmQ2N2FkMjllMTE4NjlhMTgyND9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPWF0dGFjaG1lbnQlM0IlMjBmaWxlbmFtZSUzRCUyMjUxM

### Model as you like

In [6]:
#@markdown #### MODEL MUST BE FINETUNED FROM STABLE DIFFUSION V2.0 OR ITS DESCENDANTS!!
SPECIFIED_MODEL_URL = "https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/wd-1-4-anime_e2.ckpt"  # @param {type: "string"}
SPECIFIED_MODEL_NAME = SPECIFIED_MODEL_URL.split("/")[-1]
SPECIFIED_MODEL_PATH = f"/content/stable-diffusion-webui/models/Stable-diffusion/{SPECIFIED_MODEL_NAME}"

!wget {SPECIFIED_MODEL_URL} -O {SPECIFIED_MODEL_PATH}

--2023-01-20 06:25:38--  https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/wd-1-4-anime_e2.ckpt
Resolving huggingface.co (huggingface.co)... 54.235.118.239, 3.231.67.228, 2600:1f18:147f:e800:671:b733:ecf3:a585, ...
Connecting to huggingface.co (huggingface.co)|54.235.118.239|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/79/67/7967022b375f7b6dc4e08fbd7a59bec0715529069c44ed775306c857b734e0af/c76e0962bc60ccdc18e0dce387635b472b5a19038d637216030acbbe6eda2713?response-content-disposition=attachment%3B%20filename%3D%22wd-1-4-anime_e2.ckpt%22&Expires=1674455138&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzc5LzY3Lzc5NjcwMjJiMzc1ZjdiNmRjNGUwOGZiZDdhNTliZWMwNzE1NTI5MDY5YzQ0ZWQ3NzUzMDZjODU3YjczNGUwYWYvYzc2ZTA5NjJiYzYwY2NkYzE4ZTBkY2UzODc2MzViNDcyYjVhMTkwMzhkNjM3MjE2MDMwYWNiYmU2ZWRhMjcxMz9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPWF0dGFjaG1lbnQlM0IlMjBmaWxlbmFtZSUzRCUyMndk

## 2.2 Perform Task Operations

In [7]:
# @markdown ### Optional Configuration

# @markdown ---

# @markdown #### 1. VAE replacement
# @markdown If you wanna replace original SD's VAE with specified one, keep it checked
REPLACE_VAE = True  # @param {type: "boolean"}

# @markdown ---

# @markdown #### 2. Set multipliers for task vectors
# @markdown If `DEPTH = 0.5` and `SPECIFIED_MODEL = 0.1`, the outcome WILL NOT reflect effects of specified model.

# @markdown If `DEPTH = 0.1` and `SPECIFIED_MODEL = 0.8`, the outcome WILL reflect effects of specified model, but not depth.

# @markdown (I recommend `DEPTH = 0.5` and `SPECIFIED_MODEL = 0.75` if Waifu Diffusion)
DEPTH = 0.45  # @param {type: "number"}
SPECIFIED_MODEL = 0.7  # @param {type: "number"}

# @markdown ---
# @markdown #### 3. Set custom depth model name
# @markdown Customized depth model is stored to `/content/stable-diffusion-webui/models/Stable-diffusion/{MODEL_NAME}.ckpt`.

# @markdown Wanna use in local, you may download it and `yaml` file whose name is the same as the model.

MODEL_NAME = "custom-depth"  # @param {type: "string"}
MODEL_PATH = f"/content/stable-diffusion-webui/models/Stable-diffusion/{MODEL_NAME}.ckpt"
MODEL_YAML_PATH = f"/content/stable-diffusion-webui/models/Stable-diffusion/{MODEL_NAME}.yaml"

# load models
base = torch.load(
    "/content/stable-diffusion-webui/models/Stable-diffusion/512-base-ema.ckpt",
    weights_only=True,
    map_location="cuda",
)
task_specified = torch.load(
    SPECIFIED_MODEL_PATH, 
    weights_only=True,
    map_location="cuda",
)

# for saving RAM, remove unnecessary weights
for key in set(task_specified["state_dict"].keys()) | set(base["state_dict"].keys()):
    if "cond_stage_model" in key:
        try:
            del task_specified["state_dict"][key], base["state_dict"][key]
        except Exception:
            pass
        
    elif "model_ema" in key:
        try:
            del base["state_dict"][key]
        except Exception:
            pass

# make task vector of specified model
for key in set(task_specified["state_dict"].keys()):
    if "model.diffusion_model" in key:
        task_specified["state_dict"][key] = task_specified["state_dict"][key] - base["state_dict"][key]

depth = torch.load(
    "/content/stable-diffusion-webui/models/Stable-diffusion/512-depth-ema.ckpt",
    map_location="cpu",
)

# perform task operation
for key in set(depth["state_dict"].keys()) & set(task_specified["state_dict"].keys()):
    # replace weight of VAE with specified weight, if REPLACE_VAE is True
    if "first_stage_model" in key:
        if REPLACE_VAE:
            depth["state_dict"][key] = task_specified["state_dict"][key].cpu()
    # don't replace weight of an input block whose dimension is different from each other
    elif "model.diffusion_model.input_blocks.0.0" in key:
        pass
    # otherwise, add task weight
    elif "model.diffusion_model" in key:
        task_depth = depth["state_dict"][key] - base["state_dict"][key].cpu()
        depth["state_dict"][key] = base["state_dict"][key].cpu() + (task_depth * DEPTH + task_specified["state_dict"][key].cpu() * SPECIFIED_MODEL)

del task_specified, base
gc.collect()

# save customized model
torch.save(depth, MODEL_PATH)

del depth
gc.collect()

# make yaml file to use depth model in webui
with open(MODEL_YAML_PATH, "w") as f:
  f.write(
"""
model:
  base_learning_rate: 5.0e-07
  target: ldm.models.diffusion.ddpm.LatentDepth2ImageDiffusion
  params:
    linear_start: 0.00085
    linear_end: 0.0120
    num_timesteps_cond: 1
    log_every_t: 200
    timesteps: 1000
    first_stage_key: "jpg"
    cond_stage_key: "txt"
    image_size: 64
    channels: 4
    cond_stage_trainable: false
    conditioning_key: hybrid
    scale_factor: 0.18215
    monitor: val/loss_simple_ema
    finetune_keys: null
    use_ema: False

    depth_stage_config:
      target: ldm.modules.midas.api.MiDaSInference
      params:
        model_type: "dpt_hybrid"

    unet_config:
      target: ldm.modules.diffusionmodules.openaimodel.UNetModel
      params:
        use_checkpoint: True
        image_size: 32 # unused
        in_channels: 5
        out_channels: 4
        model_channels: 320
        attention_resolutions: [ 4, 2, 1 ]
        num_res_blocks: 2
        channel_mult: [ 1, 2, 4, 4 ]
        num_head_channels: 64 # need to fix for flash-attn
        use_spatial_transformer: True
        use_linear_in_transformer: True
        transformer_depth: 1
        context_dim: 1024
        legacy: False

    first_stage_config:
      target: ldm.models.autoencoder.AutoencoderKL
      params:
        embed_dim: 4
        monitor: val/rec_loss
        ddconfig:
          #attn_type: "vanilla-xformers"
          double_z: true
          z_channels: 4
          resolution: 256
          in_channels: 3
          out_ch: 3
          ch: 128
          ch_mult:
            - 1
            - 2
            - 4
            - 4
          num_res_blocks: 2
          attn_resolutions: [ ]
          dropout: 0.0
        lossconfig:
          target: torch.nn.Identity

    cond_stage_config:
      target: ldm.modules.encoders.modules.FrozenOpenCLIPEmbedder
      params:
        freeze: True
        layer: "penultimate"
"""
  )

# remove models
os.remove("/content/stable-diffusion-webui/models/Stable-diffusion/512-base-ema.ckpt")
os.remove("/content/stable-diffusion-webui/models/Stable-diffusion/512-depth-ema.ckpt")
os.remove(SPECIFIED_MODEL_PATH)

# 3. Launch WebUI

## 3.1 Crash it!

To free RAM and VRAM used for task operation, The next cell will force the runtime to crash deliberately.

You DON'T need to restart and re-execute cells above. After crashed, just execute the next cell.

In [ ]:
os.kill(os.getpid(), 9)

## 3.2 Run AUTOMATIC1111's WebUI

WebUI's URL follows `Running on public URL`.

---

#### Some instructs about prompting

Using Waifu Diffusion V1.4's task vector, you may get better results if use `((masterpiece, best quality))` as prefix of prompt.

And here is one example of negative prompt. One day I picked this up somewhere:
```
lowres, ((bad anatomy)), ((bad hands)), text, missing finger, extra digits, fewer digits, blurry, ((mutated hands and fingers)), (poorly drawn face), ((mutation)), ((deformed face)), (ugly), ((bad proportions)), ((extra limbs)), extra face, (double head), (extra head), ((extra feet)), monster, logo, cropped, worst quality, low quality, normal quality, jpeg, humpbacked, long body, long neck, ((jpeg artifacts)), ((bad composition))
```

In [1]:
# use CLIP's penultimate layer
import json
CONFIG_PATH = "/content/stable-diffusion-webui/config.json"
with open(CONFIG_PATH, "r") as f:
    conf = json.loads(f.read())
conf["CLIP_stop_at_last_layers"] = 2
with open(CONFIG_PATH, "w") as f:
    json.dump(conf, f)

# run webui
%cd /content/stable-diffusion-webui
!COMMANDLINE_ARGS="--share --gradio-debug --no-half-vae" REQS_FILE="requirements.txt" python launch.py

/content/stable-diffusion-webui
Python 3.8.10 (default, Nov 14 2022, 12:59:47) 
[GCC 9.4.0]
Commit hash: b165e341e70300f66c8a77b9383041f4147b24a8
Installing requirements for Web UI
Launching Web UI with arguments: --share --gradio-debug --no-half-vae
No module 'xformers'. Proceeding without it.
Checkpoint custom-depth.ckpt [299caa9f29] not found; loading fallback custom-depth.ckpt
Loading config from: /content/stable-diffusion-webui/models/Stable-diffusion/custom-depth.yaml
LatentDepth2ImageDiffusion: Running in eps-prediction mode
DiffusionWrapper has 865.91 M params.
Calculating sha256 for /content/stable-diffusion-webui/models/Stable-diffusion/custom-depth.ckpt: 751c87dd92eb15fe87320a44d48ba581f6a6c5e184c28d1f30d7b15b9e7ce532
Loading weights [751c87dd92] from /content/stable-diffusion-webui/models/Stable-diffusion/custom-depth.ckpt
Applying cross attention optimization (Doggettx).
Textual inversion embeddings loaded(0): 
Model loaded in 58.2s (5.0s create model, 52.1s load weights).